## 1. Create a simple neural network

In [10]:
from nn import Layer, Module
from optim import Adam
from tensor import Tensor

In [11]:
class MLP(Module):
    def __init__(self):
        super().__init__()
        self.linear_1 = Layer(3, 3)
        self.linear_2 = Layer(3, 6)
        self.linear_3 = Layer(6, 1)

    def forward(self, x):
        x = FF.tanh(self.linear_1(x))
        x = FF.tanh(self.linear_2(x))
        x = FF.tanh(self.linear_3(x))
        return x

In [12]:
model = MLP()
optim = Adam(model.parameters()) 

In [13]:
import numpy as np

X = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]

Y = [[1.0], [-1.0], [-1.0], [1.0]]

X = Tensor(X)  # Convert to tensor
Y = Tensor(Y)  # Convert to tensor (1D array)

In [14]:
def mse_loss(predictions, targets):
    return ((predictions - targets) ** 2).sum() / predictions.shape[0]

In [15]:
epochs = 10000  

for epoch in range(epochs):
    predictions = model(X)
    loss = mse_loss(predictions, Y)
    optim.zero_grad()
    loss.backward()
    optim.step() 
    print(f"Epoch {epoch}, Loss: {loss.data}")

Epoch 0, Loss: 1.9216758012771606
Epoch 1, Loss: 1.9076101779937744
Epoch 2, Loss: 1.89114248752594
Epoch 3, Loss: 1.8719196319580078
Epoch 4, Loss: 1.8495622873306274
Epoch 5, Loss: 1.8236700296401978
Epoch 6, Loss: 1.7938300371170044
Epoch 7, Loss: 1.759642481803894
Epoch 8, Loss: 1.7207039594650269
Epoch 9, Loss: 1.6765369176864624
Epoch 10, Loss: 1.6266677379608154
Epoch 11, Loss: 1.5707229375839233
Epoch 12, Loss: 1.5085744857788086
Epoch 13, Loss: 1.4405536651611328
Epoch 14, Loss: 1.367582082748413
Epoch 15, Loss: 1.2912013530731201
Epoch 16, Loss: 1.2137178182601929
Epoch 17, Loss: 1.1384141445159912
Epoch 18, Loss: 1.0694339275360107
Epoch 19, Loss: 1.011193037033081
Epoch 20, Loss: 0.9673327803611755
Epoch 21, Loss: 0.9394325613975525
Epoch 22, Loss: 0.9260105490684509
Epoch 23, Loss: 0.9227170348167419
Epoch 24, Loss: 0.9238867163658142
Epoch 25, Loss: 0.9242365956306458
Epoch 26, Loss: 0.9199195504188538
Epoch 27, Loss: 0.9087940454483032
Epoch 28, Loss: 0.8901950716972351


## Example 2: Comparison between derivative calculation in PyTorch and MinTorch

In [72]:
import torch
import nn.functional as FF

import torch.nn.functional as F


def arrays_are_close(arr_1, arr_2):
    print(np.allclose(arr_1, arr_2, atol=1e-5))

val_a = [
    [1, 2, 3, 4],
    [4, 5, 6, 10],
    [9, -1, 1, 1]
]

val_b = [
    [1, -2, 3, 4],
    [4, 4, -6, 10],
    [1, -1, 0, 1]
]
val_q = [
    [2, 2, 2],
    [2, 2, 2],
    [2, 2, 2],
    [2, 2, 2],
    [2, 2, 2]
]

a_torch = torch.tensor(val_a, requires_grad=True, dtype=torch.float32)
b_torch = torch.tensor(val_b, requires_grad=True, dtype=torch.float32)
q_torch = torch.tensor(val_q, requires_grad=True, dtype=torch.float32)
c_torch = F.linear(a_torch, b_torch)
d_torch = F.linear(c_torch, q_torch)
e_torch = d_torch.sum()
e_torch.backward()


a_mtorch = Tensor(val_a)
b_mtorch = Tensor(val_b)
c_mtorch = FF.linear(a_mtorch, b_mtorch)
q_mtorch = Tensor(val_q)
d_mtorch = FF.linear(c_mtorch, q_mtorch)
e_mtorch = d_mtorch.sum()
e_mtorch.backward()




arrays_are_close(a_torch.grad.detach().numpy(), a_mtorch.grad) # compare de/da
arrays_are_close(b_torch.grad.detach().numpy(), b_mtorch.grad) # compare de/da
arrays_are_close(c_torch.grad.detach().numpy(), c_mtorch.grad) # compare de/da
arrays_are_close(d_torch.grad.detach().numpy(), d_mtorch.grad) # compare de/da
arrays_are_close(e_torch.grad.detach().numpy(), e_mtorch.grad) # compare de/da
arrays_are_close(q_torch.grad.detach().numpy(), q_mtorch.grad) # compare de/da



True
True


/var/folders/q4/hx30bhz10lgchmwl379mfn4m0000gn/T/ipykernel_77058/2195455619.py:51: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1670525699189/work/build/aten/src/ATen/core/TensorBody.h:485.)
  arrays_are_close(c_torch.grad.detach().numpy(), c_mtorch.grad) # compare de/da


AttributeError: 'NoneType' object has no attribute 'detach'

## Example 3: High Dimensional Case

In [75]:
A = torch.tensor([
    [
        [1, 2, 3, 4]
    ],
    [
        [1, 9, -1, 4]
    ],
    [
        [1, 2, 3, -1]
    ]
], requires_grad=True, dtype=torch.float32)

B = torch.tensor([
    
        [1, 9, 3, 4]
    ,
    
        [0, 1, -1, -11]
    ,
    
        [1, 21, 11, -1]
    
], requires_grad=True, dtype=torch.float32)

C = F.linear(A, B)

D = C.sum()
D.backward()




AA = Tensor([
    [
        [1, 2, 3, 4]
    ],
    [
        [1, 9, -1, 4]
    ],
    [
        [1, 2, 3, -1]
    ]
])

BB = Tensor([
    
        [1, 9, 3, 4]
    ,
    
        [0, 1, -1, -11]
    ,
    
        [1, 21, 11, -1]
    
])

CC = FF.linear(AA, BB)
DD = CC.sum()
DD.backward()


arrays_are_close(D.grad.detach().numpy(), DD.grad) # compare de/da




/var/folders/q4/hx30bhz10lgchmwl379mfn4m0000gn/T/ipykernel_77058/2219479040.py:62: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1670525699189/work/build/aten/src/ATen/core/TensorBody.h:485.)
  arrays_are_close(D.grad.detach().numpy(), DD.grad) # compare de/da


AttributeError: 'NoneType' object has no attribute 'detach'

## Example 4: Matrix Inversion Using a Linear Model

In [76]:
# Matrix Inverse Example
A = Tensor([
    [4.0, 7.0],
    [2.0, 6.0]
])


inverse_model = Layer(2, 2, bias=False)

def inverse_loss(A, A_inv):
    identity = Tensor(np.eye(A.shape[0]))
    return ((FF.linear(A, A_inv) - identity) ** 2).sum()

optim = Adam(inverse_model.parameters(), lr=0.01)

for epoch in range(10000):
    optim.zero_grad()
    A_inv = inverse_model(A)
    loss = inverse_loss(A, A_inv)
    loss.backward()
    optim.step()
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.data}")

# Result
print("Original Matrix:\n", A.data)
print("Calculated Inverse:\n", inverse_model.weight.data)
print("Product Result:\n", FF.linear(A, A_inv))



Epoch 0, Loss: 7495.48876953125
Epoch 100, Loss: 49.809471130371094
Epoch 200, Loss: 41.146358489990234
Epoch 300, Loss: 32.60671615600586
Epoch 400, Loss: 24.57964515686035
Epoch 500, Loss: 17.695505142211914
Epoch 600, Loss: 12.199546813964844
Epoch 700, Loss: 8.074848175048828
Epoch 800, Loss: 5.1494951248168945
Epoch 900, Loss: 3.183157444000244
Epoch 1000, Loss: 1.9285763502120972
Epoch 1100, Loss: 1.1682037115097046
Epoch 1200, Loss: 0.7302556037902832
Epoch 1300, Loss: 0.49038660526275635
Epoch 1400, Loss: 0.36520031094551086
Epoch 1500, Loss: 0.3025755286216736
Epoch 1600, Loss: 0.27206701040267944
Epoch 1700, Loss: 0.2570319175720215
Epoch 1800, Loss: 0.248954638838768
Epoch 1900, Loss: 0.24375925958156586
Epoch 2000, Loss: 0.2396230548620224
Epoch 2100, Loss: 0.2357858419418335
Epoch 2200, Loss: 0.23195256292819977
Epoch 2300, Loss: 0.22801710665225983
Epoch 2400, Loss: 0.22394447028636932
Epoch 2500, Loss: 0.21972452104091644
Epoch 2600, Loss: 0.21535536646842957
Epoch 2700,